In [1]:
# min(clip(r(theta), 1-ε, 1+ε) * A(s, a), r(theta) * A(s, a))
# for autoformatting
# %load_ext jupyter_black

第一行 !apt-get update && apt-get install ffmpeg freeglut3-dev xvfb 用於更新套件管理器和安裝所需的軟體包。這裡安裝了 ffmpeg、freeglut3-dev 和 xvfb，這些套件在視覺化時可能會被使用到。

第二行 !pip install "stable-baselines3[extra]>=2.0.0a4" 則是使用 pip 命令安裝 stable-baselines3 套件，並指定了版本 >=2.0.0a4。同時，方括號中的 extra 表示安裝 stable-baselines3 的額外功能模組。

In [2]:
!apt-get update && apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 108 kB in 1s (80.0 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg

In [3]:
import stable_baselines3
print(f"{stable_baselines3.__version__=}")

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


stable_baselines3.__version__='2.0.0a13'


## Imports

In [4]:
import gymnasium as gym
import numpy as np

print(f"{gym.__version__=}")

gym.__version__='0.28.1'


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
!pip install gym ufal.pybox2d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


匯入強化學習（Reinforcement Learning）模型

導入了 stable_baselines3 中的 PPO 模型。

In [6]:
from stable_baselines3 import PPO

從 stable_baselines3.ppo 模塊中導入 MlpPolicy 類。

In [7]:
from stable_baselines3.ppo import MlpPolicy

創建了一個名為 env 的環境，使用 "LunarLander-v2" 遊戲，並設置 render_mode 參數為 "rgb_array"，這將在後續的渲染中返回 RGB 數組。

接著創建了一個 PPO 模型，使用 MlpPolicy 作為策略網絡，在環境 env 上進行訓練。 verbose=0 參數表示在訓練過程中不輸出冗餘信息。

In [8]:
# 1
env = gym.make("LunarLander-v2", render_mode="rgb_array")
model = PPO(MlpPolicy, env, verbose=0)

從 stable_baselines3.common.evaluation 模塊中導入 evaluate_policy 函數。

In [9]:
# 2
from stable_baselines3.common.evaluation import evaluate_policy

創建了一個獨立的評估環境 eval_env，使用 "LunarLander-v2" 遊戲，並設置 render_mode 參數為 "rgb_array"，這將在後續的渲染中返回 RGB 數組。

接著使用 evaluate_policy 函數對未經訓練的模型 model 在評估環境 eval_env 上進行評估，評估次數為 100 次。

最後，將評估結果打印輸出，包括平均獎勵 mean_reward 和獎勵的標準差 std_reward。

In [10]:
# 3
# Use a separate environement for evaluation
eval_env = gym.make("LunarLander-v2", render_mode="rgb_array")

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:-271.83 +/- 99.87


## Train the agent and evaluate it

對智能體進行訓練，訓練步數為 100 步，使用 model.learn(total_timesteps=100) 函數進行訓練。

In [11]:
# Train the agent for 100 steps
model.learn(total_timesteps=100)

對訓練過的智能體進行評估。

使用 evaluate_policy 函數對訓練後的模型 model 在評估環境 eval_env 上進行評估，評估次數為 100 次。

最後，將評估結果打印輸出，包括平均獎勵 mean_reward 和獎勵的標準差 std_reward。

Apparently the training went well, the mean reward increased a lot ! 

### Prepare video recording

In [12]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [13]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

使用VecVideoRecorder包裝器記錄視頻

In [14]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv


def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make("LunarLander-v2", render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

### Visualize trained agent



In [15]:
record_video("LunarLander-v2", model, video_length=9000, prefix="ppo-LunarLander")

Saving video to /content/videos/ppo-LunarLander-step-0-to-step-9000.mp4
Moviepy - Building video /content/videos/ppo-LunarLander-step-0-to-step-9000.mp4.
Moviepy - Writing video /content/videos/ppo-LunarLander-step-0-to-step-9000.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/ppo-LunarLander-step-0-to-step-9000.mp4


In [17]:
show_videos("/content/videos", prefix="ppo")

Output hidden; open in https://colab.research.google.com to view.